In [1]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn
%matplotlib inline

# Day 6: Chronal Coordinates

The device on your wrist beeps several times, and once again you feel like you're falling.

"Situation critical," the device announces. "Destination indeterminate. Chronal interference detected. Please specify new target coordinates."

The device then produces a list of coordinates (your puzzle input). Are they places it thinks are safe or dangerous? It recommends you check manual page 729. The Elves did not give you a manual.

If they're dangerous, maybe you can minimize the danger by finding the coordinate that gives the largest distance from the other points.

Using only the Manhattan distance, determine the area around each coordinate by counting the number of integer X,Y locations that are closest to that coordinate (and aren't tied in distance to any other coordinate).

Your goal is to find the size of the largest area that isn't infinite. For example, consider the following list of coordinates:

```
1, 1
1, 6
8, 3
3, 4
5, 5
8, 9
```

If we name these coordinates A through F, we can draw them on a grid, putting 0,0 at the top left:

```
..........
.A........
..........
........C.
...D......
.....E....
.B........
..........
..........
........F.
```

This view is partial - the actual grid extends infinitely in all directions. Using the Manhattan distance, each location's closest coordinate can be determined, shown here in lowercase:

```
aaaaa.cccc
aAaaa.cccc
aaaddecccc
aadddeccCc
..dDdeeccc
bb.deEeecc
bBb.eeee..
bbb.eeefff
bbb.eeffff
bbb.ffffFf
```

Locations shown as . are equally far from two or more coordinates, and so they don't count as being closest to any.

In this example, the areas of coordinates A, B, C, and F are infinite - while not shown here, their areas extend forever outside the visible grid. However, the areas of coordinates D and E are finite: D is closest to 9 locations, and E is closest to 17 (both including the coordinate's location itself). Therefore, in this example, the size of the largest area is 17.

What is the size of the largest area that isn't infinite?

In [2]:
import re

lines = open("6.txt", "r").read().split('\n')
lines = [re.match("(\d*), (\d*)", l).groups() for l in lines]
lines = [tuple(int(i) for i in l) for l in lines]

coords = pd.DataFrame(lines)
coords.columns = ['x', 'y']
#coords

In [3]:
from scipy.spatial.distance import cityblock

In [4]:
x_min = coords.x.min()
x_max = coords.x.max() + 1
y_min = coords.y.min()
y_max = coords.y.max() + 1

In [5]:
data = [(letter, (s.x, s.y)) for letter, s in coords.iterrows()]

grid = {}

for x in range(x_min, x_max + 1):
    print(x)
    for y in range(y_min, y_max + 1):
        m = min(abs(x - s.x) + abs(y - s.y) for letter, s in coords.iterrows())
        for letter, (sx, sy) in data:
            if abs(x - sx) + abs(y - sy) == m:
                if grid.get((x, y), -1) != -1:
                    grid[x, y] = -1
                    break
                grid[x, y] = letter

40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304


In [6]:
grid_df = pd.DataFrame([(k[0], k[1], v) for k, v in grid.items()], columns=['x', 'y', 'letter'])
grid_df = grid_df.set_index(['x', 'y']).letter.unstack()

In [7]:
infinite = set(grid_df.iloc[0, :]) | set(grid_df.iloc[-1, :]) | set(grid_df.iloc[:, 0]) | set(grid_df.iloc[:, -1])

In [8]:
largest = grid_df.stack().value_counts()
pd.Series({k: v for k, v in largest.items() if k not in infinite})

7     4060
22    3899
39    3102
24    3063
35    3004
32    2778
20    2776
5     2381
17    2380
46    2308
49    2292
13    2150
12    1843
30    1819
25    1688
40    1659
11    1603
27    1569
14    1448
1     1417
0     1377
16    1300
21    1144
29    1113
23     839
9      756
48     734
36     465
10      73
dtype: int64

# Part Two

On the other hand, if the coordinates are safe, maybe the best you can do is try to find a region near as many coordinates as possible.

For example, suppose you want the sum of the Manhattan distance to all of the coordinates to be less than 32. For each location, add up the distances to all of the given coordinates; if the total of those distances is less than 32, that location is within the desired region. Using the same coordinates as above, the resulting region looks like this:

```
..........
.A........
..........
...###..C.
..#D###...
..###E#...
.B.###....
..........
..........
........F.
```

In particular, consider the highlighted location 4,3 located at the top middle of the region. Its calculation is as follows, where abs() is the absolute value function:

```
Distance to coordinate A: abs(4-1) + abs(3-1) =  5
Distance to coordinate B: abs(4-1) + abs(3-6) =  6
Distance to coordinate C: abs(4-8) + abs(3-3) =  4
Distance to coordinate D: abs(4-3) + abs(3-4) =  2
Distance to coordinate E: abs(4-5) + abs(3-5) =  3
Distance to coordinate F: abs(4-8) + abs(3-9) = 10
Total distance: 5 + 6 + 4 + 2 + 3 + 10 = 30
```

Because the total distance to all coordinates (30) is less than 32, the location is within the region.

This region, which also includes coordinates D and E, has a total size of 16.

Your actual region will need to be much larger than this example, though, instead including all locations with a total distance of less than 10000.

What is the size of the region containing all locations which have a total distance to all given coordinates of less than 10000?

In [9]:
grid = {}

for x in range(x_min, x_max + 1):
    for y in range(y_min, y_max + 1):
        s = 0
        for letter, (sx, sy) in data:
            s += abs(x - sx) + abs(y - sy)
        grid[x, y] = s

In [10]:
grid_df = pd.DataFrame([(k[0], k[1], v) for k, v in grid.items()], columns=['x', 'y', 'letter'])

In [11]:
len(grid_df[grid_df.letter < 10000])

36136